In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# download from here https://www.aaii.com/sentimentsurvey/sent_results
raw_df = pd.read_excel('D:\jupyter notebooks\sentiment.xlsx', skiprows=1)

# Combine first two rows to create proper column names
col_names = []
for col in range(raw_df.shape[1]):
    col_name_parts = [str(raw_df.iloc[0, col]), str(raw_df.iloc[1, col])]
    # Filter out 'nan' and 'Unnamed' parts
    col_name_parts = [part for part in col_name_parts if part.lower() != 'nan' and 'unnamed' not in part.lower()]
    col_names.append(' '.join(col_name_parts).strip())

# Now read the data skipping the first few rows and apply the combined column names
df = pd.read_excel('D:\jupyter notebooks\sentiment.xlsx', skiprows=6)
df.columns = col_names

# Clean up column names: remove any remaining 'Unnamed' references
df.columns = [col.strip() for col in df.columns]
df.head()

In [ ]:
df.rename(columns={"Reported Date": "time"}, inplace=True)

In [ ]:
df = df[pd.to_datetime(df['time'], errors='coerce').notna()]
df

In [ ]:
df=df.drop(columns=['Total'])

In [ ]:
df.columns

In [ ]:
df.rename(columns={
    'Bullish': 'bullish',
    'Neutral': 'neutral',
    'Bearish': 'bearish',
    '8-week Mov Avg': '8_week_avg',
    'Bull-Bear Spread': 'bull_bear_spread',
    'Bullish Average': 'bullish_avg',
    'Average +St. Dev.': 'bullish_avg_plus_std',
    'Average - St. Dev.': 'bullish_avg_minus_std',
    'Weekly High': 'weekly_high',
    'Weekly Low': 'weekly_low',
    'Weekly Close': 'weekly_close'
}, inplace=True)

In [ ]:
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=('Weekly Close vs Bullish Sentiment',
                                  'Weekly Close vs Bullish Sentiment with 8 week Moving Average'),
                    vertical_spacing=0.1,
                    row_heights=[0.4, 0.6],
                    specs=[[{"secondary_y": True}],  # Enable secondary y-axis for first subplot
                          [{"secondary_y": True}]])  # No secondary y-axis for second subplot

# First subplot: Weekly Close and Bullish Sentiment
fig.add_trace(
    go.Scatter(x=df['time'], y=df['weekly_close'],
               name='Weekly Close', line=dict(color='blue')),
    row=1, col=1,
    secondary_y=False  # Use primary y-axis for price
)
fig.add_trace(
    go.Scatter(x=df['time'], y=df['bullish'],
               name='Bullish Sentiment', line=dict(color='green')),
    row=1, col=1,
    secondary_y=True  # Use secondary y-axis for sentiment
)
fig.add_trace(
    go.Scatter(x=df['time'], y=df['bullish_avg'],
               name='Bullish Average', line=dict(color='red', dash='dash')),
    row=1, col=1,
    secondary_y=True  # Use secondary y-axis for sentiment average
)

fig.add_trace(
    go.Scatter(x=df['time'], y=df['bullish_avg_plus_std'],
               name='Bullish Average +StDev', line=dict(color='yellow', dash='dash')),
    row=1, col=1,
    secondary_y=True  # Use secondary y-axis for sentiment average
)

fig.add_trace(
    go.Scatter(x=df['time'], y=df['bullish_avg_minus_std'],
               name='Bullish Average -StDev', line=dict(color='orange', dash='dash')),
    row=1, col=1,
    secondary_y=True  # Use secondary y-axis for sentiment average
)

# second plot
fig.add_trace(
    go.Scatter(x=df['time'], y=df['weekly_close'],
               name='Weekly Close', line=dict(color='blue')),
    row=2, col=1,
    secondary_y=False  # Use primary y-axis for price
)
fig.add_trace(
    go.Scatter(x=df['time'], y=df['8_week_avg'],
               name='8-Week Moving Average', line=dict(color='green')),
    row=2, col=1,
    secondary_y=True  # Use secondary y-axis for moving average
)
fig.add_trace(
    go.Scatter(x=df['time'], y=df['bullish_avg'],
               name='Bullish Average', line=dict(color='red', dash='dash')),
    row=2, col=1,
    secondary_y=True  # Use secondary y-axis for sentiment average
)

fig.add_trace(
    go.Scatter(x=df['time'], y=df['bullish_avg_plus_std'],
               name='Bullish Average +StDev', line=dict(color='yellow', dash='dash')),
    row=2, col=1,
    secondary_y=True  # Use secondary y-axis for sentiment average
)

fig.add_trace(
    go.Scatter(x=df['time'], y=df['bullish_avg_minus_std'],
               name='Bullish Average -StDev', line=dict(color='orange', dash='dash')),
    row=2, col=1,
    secondary_y=True  # Use secondary y-axis for sentiment average
)


# Update layout
fig.update_layout(
    height=1200,
    width=1400,
    title_text="AAII Sentiment Analysis",
    showlegend=True
)

# Update axes titles
fig.update_yaxes(title_text="Price", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Bullish Sentiment %", row=1, col=1, secondary_y=True)
fig.update_yaxes(title_text="Price", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

# Update x-axes to show date in a readable format
fig.update_xaxes(tickformat="%Y-%m-%d")

fig.show()

# Print current readings
print("\nCurrent Readings:")
print(f"Weekly Close: ${df['weekly_close'].iloc[0]:.2f}")
print(f"Bullish Sentiment: {df['bullish'].iloc[0]:.1f}%")
print(f"8-Week Moving Average: {df['8_week_avg'].iloc[0]:.1f}%")
print(f"Bullish Average: {df['bullish_avg'].iloc[0]:.1f}%")
print(f"Average +St. Dev: {df['bullish_avg_plus_std'].iloc[0]:.1f}%")
print(f"Average -St. Dev: {df['bullish_avg_minus_std'].iloc[0]:.1f}%")